In [2]:
import requests
import pandas as pd
import curl
import datetime as dt
import re
import gspread
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials
import numpy as np
import gspread_dataframe as gd
from gspread_dataframe import set_with_dataframe

def extract_data():
    ## calling the API to load dataset
    df = pd.DataFrame()
    for i in range(1,20):
        !curl --header "PRIVATE-TOKEN:<private token>" "https://<gitlab url>/api/v4/projects/<no.>/issues?pagination=keyset&per_page=100&page={i}&sort=asc" > data.json
        data = pd.read_json('data.json')
        df = df.append(data)

    return(df)

def transform_data(df):
    ## Data Preprocessing 

    # filtering out columns that are not required
    df1 = df[['iid', 'title', 'description', 'state',
               'created_at', 'updated_at', 'closed_at', 'closed_by', 'labels',
               'milestone', 'author', 'assignee',
               'time_stats', 'task_completion_status', 'has_tasks']]

    ## flattening the dataframe

    ## flattening individual columns
    df_closed_by = df1["closed_by"].apply(pd.Series).drop(columns=["id","username","state","avatar_url","web_url"]).rename(columns={"name":"closed_by_name"})
    df_milestone = df1["milestone"].apply(pd.Series).drop(columns=['id', 'iid', 'project_id', 'description', 'state',
               'created_at', 'updated_at', 'due_date', 'start_date', 'expired',
               'web_url', 'group_id']).rename(columns={"title":"milestone_title"})
    df_author = df1["author"].apply(pd.Series).drop(columns=["id","username","state","avatar_url","web_url"]).rename(columns={"name":"author_name"})
    df_assignee = df1["assignee"].apply(pd.Series).drop(columns=["id","username","state","avatar_url","web_url"]).rename(columns={"name":"assignee_name"})
    df_time_stats = df1["time_stats"].apply(pd.Series).drop(columns=["human_time_estimate","human_total_time_spent"])
    df_task_completion_status = df1["task_completion_status"].apply(pd.Series).rename(columns={"count":"task_count","completed_count":"completed_task_count"})

    ## combining into single df
    df2 = pd.concat([df1,df_closed_by,df_milestone,df_author,df_assignee,df_time_stats,df_task_completion_status],axis=1).drop(columns=["closed_by","milestone","author","assignee","time_stats","task_completion_status"])

    ## converting time from seconds to hours
    df2["time_estimate_hours"] = df2["time_estimate"]/3600
    df2["time_spent_hours"] = df2["total_time_spent"]/3600

    #calculating ticket closure time
    df2["created_at"] = pd.to_datetime(df2["created_at"], utc=True)
    df2["creation_date"] = df2["created_at"].dt.date

    df2["closed_at"] = pd.to_datetime(df2["closed_at"], utc=True)
    df2["closed_date"] = df2["closed_at"].dt.date

    df2["closure_duration"] = df2["closed_date"]-df2["creation_date"]

    ## extracting status and duplicate labels
    df2["state"] = df2["state"].str.replace('opened','Open')
    df2["labels"] = df2["labels"].astype(str)

    status_label = ['Icebox', 'Scheduled', 'Development','MR Review','Awaiting Delivery / Testing','Delivered / QA Approved', 'Blocked']
    duplicate_label = ['Duplicate']

    conditions_status = list(map(df2['labels'].str.contains, status_label))
    conditions_duplicate = list(map(df2['labels'].str.contains, duplicate_label))

    df2['Ticket_Type'] = np.select(conditions_duplicate, duplicate_label)
    df2['Labels'] = np.select(conditions_status, status_label, 'Open')
    
    ## renaming milestone titles 
    df2["milestone_title"] = df2["milestone_title"].str.replace('V0.6','0.6')
    df2["milestone_title"] = df2["milestone_title"].str.replace('V0.6.3','0.6.3')
    df2["milestone_title"] = df2["milestone_title"].str.replace('V0.6.4','0.6.4')
    
    ##
    df2['closure_duration'].fillna(pd.Timedelta(seconds=0), inplace=True)
    df2["closure_duration"] = df2["closure_duration"].astype(int)/(24*60*60*(10**9))
    
    ## resetting the index
    df2 = df2.reset_index()
    df2 = df2.drop(columns=["index"])
    
    return(df2)

def load_data(df2):
    ## writing the data to google sheets
    scope = ['https://spreadsheets.google.com/feeds',
             'https://www.googleapis.com/auth/drive']
    credentials = ServiceAccountCredentials.from_json_keyfile_name(
        'xxxxx-xx-xxxxxxxxx-xxxxxxxxxx.json', scope)
    gc = gspread.authorize(credentials)
    

    spreadsheet_key = '<spreadsheet key>'
    wks_name = '<worksheet name>'
    d2g.upload(df2, spreadsheet_key, wks_name, credentials=credentials, row_names=True)
    
    return("ETL process complete.")

def load_data_to_historical(df2):
    ## appending data in historical google sheet
    scope = ['https://spreadsheets.google.com/feeds',
             'https://www.googleapis.com/auth/drive']
    credentials = ServiceAccountCredentials.from_json_keyfile_name(
        'xxxxx-xx-xxxxxxxxx-xxxxxxxxxx.json', scope)
    gc = gspread.authorize(credentials)

    df2 = df2[['iid','title','state','created_at','updated_at','closed_at','labels','milestone_title','author_name','assignee_name','time_estimate','total_time_spent','time_estimate_hours','time_spent_hours','creation_date','closed_date','closure_duration','Ticket_Type','Labels']]
        
    sheet = gc.open_by_key('<spreadsheet key>')
    ws = sheet.get_worksheet(0)
    ws.add_rows(df2.shape[0])
    
    start_index = len(ws.get_all_values(major_dimension='rows'))-1
    df2.index = df2.index + start_index
    
    df2['data_date'] = pd.to_datetime('now').date()
    
    cell_num = len(ws.get_all_values(major_dimension='rows'))
    cell_val = ws.cell(cell_num,22).value
    df2['week_number'] = int(cell_val)+1
    
    gd.set_with_dataframe(worksheet=ws,dataframe=df2,include_index=True,include_column_header=False,row=len(ws.get_all_values(major_dimension='rows'))+1,resize=False)

def print_df(df2):
    display(df2)
    
def main():
    
    api_df = extract_data()
    processed_df = transform_data(api_df)
    load_data(processed_df)
    load_data_to_historical(processed_df)
#     print_df(processed_df)
#     dataframe.head()
    
main()


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  285k    0  285k    0     0   214k      0 --:--:--  0:00:01 --:--:--  214k  0 64323    0     0  50409      0 --:--:--  0:00:01 --:--:-- 50370
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

<ipython-input-2-3ac6a0bf2d1d>:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data)


100  499k    0  499k    0     0   398k      0 --:--:--  0:00:01 --:--:--  398k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

<ipython-input-2-3ac6a0bf2d1d>:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data)


100  568k    0  568k    0     0   473k      0 --:--:--  0:00:01 --:--:--  473k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

...

<ipython-input-2-3ac6a0bf2d1d>:40: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_assignee = df1["assignee"].apply(pd.Series).drop(columns=["id","username","state","avatar_url","web_url"]).rename(columns={"name":"assignee_name"})
<ipython-input-2-3ac6a0bf2d1d>:40: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_assignee = df1["assignee"].apply(pd.Series).drop(columns=["id","username","state","avatar_url","web_url"]).rename(columns={"name":"assignee_name"})
<ipython-input-2-3ac6a0bf2d1d>:40: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_assignee = df1["assignee"].apply(pd.Series).drop(columns=["id","username","state","avatar_url","web_u

<ipython-input-2-3ac6a0bf2d1d>:40: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_assignee = df1["assignee"].apply(pd.Series).drop(columns=["id","username","state","avatar_url","web_url"]).rename(columns={"name":"assignee_name"})
<ipython-input-2-3ac6a0bf2d1d>:40: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_assignee = df1["assignee"].apply(pd.Series).drop(columns=["id","username","state","avatar_url","web_url"]).rename(columns={"name":"assignee_name"})
<ipython-input-2-3ac6a0bf2d1d>:40: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_assignee = df1["assignee"].apply(pd.Series).drop(columns=["id","username","state","avatar_url","web_u

<ipython-input-2-3ac6a0bf2d1d>:40: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_assignee = df1["assignee"].apply(pd.Series).drop(columns=["id","username","state","avatar_url","web_url"]).rename(columns={"name":"assignee_name"})
<ipython-input-2-3ac6a0bf2d1d>:40: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_assignee = df1["assignee"].apply(pd.Series).drop(columns=["id","username","state","avatar_url","web_url"]).rename(columns={"name":"assignee_name"})
<ipython-input-2-3ac6a0bf2d1d>:40: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_assignee = df1["assignee"].apply(pd.Series).drop(columns=["id","username","state","avatar_url","web_u

<ipython-input-2-3ac6a0bf2d1d>:40: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_assignee = df1["assignee"].apply(pd.Series).drop(columns=["id","username","state","avatar_url","web_url"]).rename(columns={"name":"assignee_name"})
<ipython-input-2-3ac6a0bf2d1d>:40: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_assignee = df1["assignee"].apply(pd.Series).drop(columns=["id","username","state","avatar_url","web_url"]).rename(columns={"name":"assignee_name"})
<ipython-input-2-3ac6a0bf2d1d>:40: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_assignee = df1["assignee"].apply(pd.Series).drop(columns=["id","username","state","avatar_url","web_u

<ipython-input-2-3ac6a0bf2d1d>:74: FutureWarning: The default value of regex will change from True to False in a future version.
  df2["milestone_title"] = df2["milestone_title"].str.replace('V0.6','0.6')
<ipython-input-2-3ac6a0bf2d1d>:75: FutureWarning: The default value of regex will change from True to False in a future version.
  df2["milestone_title"] = df2["milestone_title"].str.replace('V0.6.3','0.6.3')
<ipython-input-2-3ac6a0bf2d1d>:76: FutureWarning: The default value of regex will change from True to False in a future version.
  df2["milestone_title"] = df2["milestone_title"].str.replace('V0.6.4','0.6.4')
<ipython-input-2-3ac6a0bf2d1d>:130: FutureWarning: The parsing of 'now' in pd.to_datetime without `utc=True` is deprecated. In a future version, this will match Timestamp('now') and Timestamp.now()
  df2['data_date'] = pd.to_datetime('now').date()
<ipython-input-2-3ac6a0bf2d1d>:130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
T